# Solar-related cyclicity over the Holocene

## Authors

Deborah Khider
<a href="https://orcid.org/0000-0001-7501-8430" target="_blank" rel="noopener noreferrer">
  <img src="https://orcid.org/sites/default/files/images/orcid_16x16.png" alt="ORCID iD" style="vertical-align: text-bottom;"/>
</a>

## Preamble

Centennial to millennial quasi-periodic oscillations appear in all manner of well-dated Holocene paleoclimate records. Though the origin of this variability is debated, the similarity between the periods recovered from the spectral analysis of Holocene paleoclimate records and corresponding reconstructions of total solar irradiance (TSI) has led to the hypothesis that TSI variations drive Holocene climate cycles (the "solar hypothesis").

In this notebook, we demonstrate the use of [the `Pyleoclim` software package](https://pyleoclim-util.readthedocs.io/en/latest/) to estimate the spectra from Holocene temperature records and estimate whether they exhibit significant periodicities in the 800-3000 year band.

<div style="
    padding: 10px; 
    background-color: #e7f3fe; 
    border-left: 6px solid #2196F3; 
    margin-bottom: 15px;">
  <strong>Note:</strong> This notebook was created with the help of an artificial intelligent (AI) assistant for paleoclimatology (PaleoPAL) that it currently (November 2025) under active development. Code and conclusions were reviewed by a human. Prompts to the AI are written in the markdown cells (preceded by @agent) and kept for transparency. Correction to the code is marked. The additional lines of code generated by PaleoPAL but not needed in the context of this notebook are commented out.
</div>

Technical skills involved:

- Loading data in the [LiPD](https://lipd.net) format using the [LiPDGraph](https://linkedearth.graphdb.mint.isi.edu).
- Applying and interpreting spectral analysis in with [`Pyleoclim`](https://pyleoclim-util.readthedocs.io/en/latest/)
- Estimating significant periodicities.


### Data

The data is from the [Temperature 12k Database](https://www.nature.com/articles/s41597-020-0530-7) by Kaufman et al. (2020), stored in the Linked PaleoData ([LiPD](https://lipd.net)) format on the [LiPDGraph](https://linkedearth.graphdb.mint.isi.edu).

### Reading time
15 min

### Keywords
Pyleoclim; LiPD; SPARQL; Holocene; Spectral Analysis.

Let's import the necessary packages:

In [1]:
# To query the graph
import json
import requests
import pandas as pd
import io

# Analysis
import pyleoclim as pyleo
import numpy as np

# To save
import pickle

## Data Retrieval

Following [this tutorial](http://linked.earth/pylipdTutorials/notebooks/L2_using_graphdb.html), we perform a [SPARQL](http://linked.earth/pylipdTutorials/graph.html#what-is-sparql) on the [LiPDGraph](https://linkedearth.graphdb.mint.isi.edu/) to assemble our database.

We need to return the following information for all the datasets included in the `Temp12k` compilation with the variable name `temperature`:

- `dataSetName`: The name of the dataset
- `archiveType`: The type of archive on which the measurements were made
- `Latitude`, `Longitude`, and `Elevation`: the geographical coordinates for the site
- `paleoData_variableName`: The name of the paleo variable as originally reported by the investigator
- `paleoData_standardName`: The standard name for the paleo variable (here, `temperature`)
- `paleoData_values`: The values for the paleo variable
- `paleoData_units`: The units associated with the paleo variable
- `paleoData_proxy`: The proxy used to infer paleoenvironmental conditions
- `paleoData_proxyGeneral`: Generalized grouping for proxy
- `TSiD`: The unique ID for the timeseries
- `time_variableName`: The name for the time variable. Since we are dealing with the common era, should be all “year”.
- `timevar_standardName`: The standard name for the time variable. Since we are dealing with the common era, should be all “year”. So let’s use this as a filter.
- `time_values`: The values for the time axis
- `time_units`: the units for the time axis.


@agent sparql Return the dataset names for the records in the Temp12k compilation

In [ ]:

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

def fetch_sparql(endpoint_url: str, query: str) -> pd.DataFrame:
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Get variable names (column names)
    cols = results["head"]["vars"]

    # Build rows
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for col in cols:
            if col in result:
                row[col] = result[col]["value"]
            else:
                row[col] = None
        rows.append(row)

    return pd.DataFrame(rows)

endpoint = "https://linkedearth.graphdb.mint.isi.edu/repositories/LiPDVerse-dynamic"
query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ex: <http://example.org/>

SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object.
  FILTER(isIRI(?subject) && isIRI(?predicate) && isLiteral(?object))
}
LIMIT 10"""
sparql_results_1760980808_8830cd46 = fetch_sparql(endpoint, query)
print(sparql_results_1760980808_8830cd46.head())
